# `重要`
在克隆本 git 项目后，如果未按照现有方式挂载 `dataspace` 数据卷，则需要按情况将每一个代码块的 `processed=True` 修改为 `processed=False`，然后运行，以将处理后的数据保存至某个文件夹（原为`dataspace/Prototype`），并在`tankbind_prototype/datasets` 下生成训练、测试用数据集。
若成功挂载 `dataspace`，请跳到本脚本的 「<font color='gold'>「从这里开始」</font>数据集构建」，并以该处开始构建数据集。

# overview

### Preperation | 路径准备


In [56]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import os
import torch
pdbbind_path = "/home/jovyan/dataspace/PDBbind2020/all_pdbbind"
proto_path = "/home/jovyan/dataspace/Prototype"
p2rank = "bash /home/jovyan/dataspace/p2rank_2.3/prank"
tankbind_src_folder_path = "../tankbind_prototype/"
import sys
sys.path.insert(0, tankbind_src_folder_path)
from feature_utils import generate_sdf_from_smiles_using_rdkit

In [57]:
internal_pdbs = ["7kac", "6scm", "7s1s", "7t47", "7kcc", "6rpg"] # 7t47对应KRAIS-G12D；7kcc对应MAT2A；6rpg对应pdl1 

### 构建数据集

In [58]:
test = "/home/jovyan/dataspace/Internal/qsar_test/all_middle.test.csv"

In [59]:
df = pd.read_csv(test)
df.target.unique()

array(['KRAS-G12D', 'MAT2A', 'PDL1', 'VEGFR2', 'BRAF', 'JAK2', 'EGFR',
       'CDK4', 'BTK', 'NTRK1', 'ALK', 'FGFR1', 'PRMT5', 'HPK1_3',
       'HPK1_4', 'SOS1'], dtype=object)

In [60]:
df = df.drop_duplicates().reset_index(drop=True)

In [61]:
if False:
    test1 = "/home/jovyan/dataspace/Internal/label/PRMT5/all_middle.test.csv"
    test2 = "/home/jovyan/dataspace/Internal/label/SOS1-HPK1/all_middle.test.csv"
    df1 = pd.read_csv(test1)
    df2 = pd.read_csv(test2)
    df1 = df1[['smiles', 'value', 'target', 'id_in_patent']]
    df2 = df2[['smiles', 'value', 'target', 'id_in_patent']]
    data = pd.concat([df1, df2]).reset_index(drop=True)
    del df1, df2
df = df[df.target.isin(['HPK1_3', 'HPK1_4', "PRMT5", 'SOS1', "KRAS-G12D", "PDL1", "MAT2A"])]
data =df[['smiles', 'value', 'target', 'id_in_patent']]
data.head(5)
sample_id = list(range(len(df)))
target = list(data.target)
sample_id = ["ID_internal_" + __ + "_" + str(_) for (_, __) in zip(sample_id, target)]
print(sample_id[:5])
data['sample_id'] = sample_id

data['pdb'] = data['target'].apply(lambda x: {"PRMT5": "7s1s", "SOS1": "6scm", "HPK1_3": "7kac", "HPK1_4": "7kac", "KRAS-G12D": "7t47", "PDL1": "6rpg", "MAT2A": "7kcc"}[x])

['ID_internal_KRAS-G12D_0', 'ID_internal_KRAS-G12D_1', 'ID_internal_KRAS-G12D_2', 'ID_internal_KRAS-G12D_3', 'ID_internal_KRAS-G12D_4']


/tmp/ipykernel_82450/2554067007.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['sample_id'] = sample_id
/tmp/ipykernel_82450/2554067007.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['pdb'] = data['target'].apply(lambda x: {"PRMT5": "7s1s", "SOS1": "6scm", "HPK1_3": "7kac", "HPK1_4": "7kac", "KRAS-G12D": "7t47", "PDL1": "6rpg", "MAT2A": "7kcc"}[x])


In [62]:
data

,smiles,value,target,id_in_patent,sample_id,pdb
0,Oc1cc(-c2ncc3c(NC4CNC4)nc(OCC45CCCN4CCC5)nc3c2...,46.5,KRAS-G12D,0,ID_internal_KRAS-G12D_0,7t47
1,CN[C@H]1C[C@@H](Nc2nc(OCC34CCCN3CCC4)nc3c(F)c(...,80.9,KRAS-G12D,1,ID_internal_KRAS-G12D_1,7t47
2,C#Cc1cccc2cccc(-c3ncc4c(NCC5(N(C)C)CCC5)nc(OC[...,22.0,KRAS-G12D,2,ID_internal_KRAS-G12D_2,7t47
3,CCc1cccc2cccc(-c3ncc4c(NCC5(N(C)C)CCC5)nc(OC[C...,59.2,KRAS-G12D,3,ID_internal_KRAS-G12D_3,7t47
4,CCc1c(F)ccc2cccc(-c3ncc4c(NCC5(N(C)C)CCC5)nc(O...,30.4,KRAS-G12D,4,ID_internal_KRAS-G12D_4,7t47
...,...,...,...,...,...,...
1884,C[C@@H](NC(=O)c1cc(N2CCn3nccc3C2)nc2cc[nH]c12)...,89.8,SOS1,114,ID_internal_SOS1_1052,6scm
1885,C[C@@H](NC(=O)c1cc(N2CCc3c(cnn3C)C2)nc2cc[nH]c...,105.5,SOS1,115,ID_internal_SOS1_1053,6scm
1886,C[C@H](NC(=O)c1cc(N2CCC(O)C2)nc2cc[nH]c12)c1cc...,109.6,SOS1,116,ID_internal_SOS1_1054,6scm
1887,COC1CCN(c2cc(C(=O)N[C@H](C)c3cccc(C(F)F)c3F)c3...,125.4,SOS1,117,ID_internal_SOS1_1055,6scm


### 为 `internal` 数据集生成小分子文件

In [64]:
processed = True
if not processed:
    os.system("rm -rf /home/jovyan/dataspace/Prototype/rdkit_generated_sdfs_for_internal_data")
    os.system("mkdir -p /home/jovyan/dataspace/Prototype/rdkit_generated_sdfs_for_internal_data")
    
    for i, line in data.iterrows():
        smiles = line['smiles']
        index = line['sample_id']
        try:
            generate_sdf_from_smiles_using_rdkit(smiles=smiles, rdkitMolFile=f"/home/jovyan/dataspace/Prototype/rdkit_generated_sdfs_for_internal_data/{index}.sdf", shift_dis=0)
        except:
            print(index, smiles)

### 构建小分子特征词典 `compound_dict`

In [65]:
from feature_utils import extract_torchdrug_feature_from_mol
from feature_utils import read_mol

processed = True

if not processed:
    compound_dict = {}
    skip_smiles_list = []
    for i, line in data.iterrows():
        smiles = line['smiles']
        index = line['sample_id']
        mol, _ = read_mol(f"/home/jovyan/dataspace/Prototype/rdkit_generated_sdfs_for_internal_data/{index}.sdf", None)
        # extract features from sdf.
        try:
            compound_dict[index] = extract_torchdrug_feature_from_mol(mol, has_LAS_mask=True)  # self-dock set has_LAS_mask to true
        except Exception as e:
            print(index, e)
            skip_smiles_list.append(index)
            
    torch.save(compound_dict, f"/home/jovyan/dataspace/Prototype/dicts/INTERNAL_compound_torchdrug_features.pt")
else:
    print("Load saved compound_dict.")
      
    compound_dict = torch.load(f"/home/jovyan/dataspace/Prototype/dicts/INTERNAL_compound_torchdrug_features.pt")

### 构建蛋白质特征词典 `protein_dict`

In [66]:
from feature_utils import get_protein_feature

processed = True
  
if not processed:
    input_ = []
    tankbind_data_path = f"/home/jovyan/dataspace/Prototype/tankbind_data"
    protein_embedding_folder = f"{tankbind_data_path}/gvp_protein_embedding_for_internal"
    os.system(f"mkdir -p {protein_embedding_folder}")
    for pdb in internal_pdbs:
        proteinFile = f"/home/jovyan/dataspace/Internal/pdbs/{pdb}.pdb"
        toFile = f"{protein_embedding_folder}/{pdb}.pt"
        x = (pdb, proteinFile, toFile)
        input_.append(x)
            

    from Bio.PDB import PDBParser
    from feature_utils import get_clean_res_list
    import torch
    torch.set_num_threads(1)

    def batch_run(x):
        protein_dict = {}
        pdb, proteinFile, toFile = x
        parser = PDBParser(QUIET=True)
        s = parser.get_structure(pdb, proteinFile)
        res_list = get_clean_res_list(s.get_residues(), verbose=False, ensure_ca_exist=True)
        protein_dict[pdb] = get_protein_feature(res_list)
        torch.save(protein_dict, toFile)
            

    import mlcrate as mlc
    import os
    pool = mlc.SuperPool(64)
    pool.pool.restart()
    _ = pool.map(batch_run,input_)
    pool.exit()


    import torch
    protein_dict = {}
    for pdb in tqdm(internal_pdbs):
        protein_dict.update(torch.load(f"{protein_embedding_folder}/{pdb}.pt"))
            
            
    torch.save(protein_dict, "/home/jovyan/dataspace/Prototype/dicts/INTERNAL_protein_dict.pt")
    
else:
    print("Load saved protein_dict.")
    protein_dict = torch.load("/home/jovyan/dataspace/Prototype/dicts/INTERNAL_protein_dict.pt")

Load saved protein_dict.


In [67]:
p2rank_prediction_folder = f"/home/jovyan/dataspace/Prototype/p2rank_results_for_internal_data"
os.system(f"mkdir -p {p2rank_prediction_folder}")
ds = f"{p2rank_prediction_folder}/protein_list.ds"
with open(ds, "w") as out:
    for pdb in internal_pdbs:
        out.write(f"../../Internal/pdbs/{pdb}.pdb\n")
cmd = f"{p2rank} predict {ds} -o {p2rank_prediction_folder} -threads 16"
os.system(cmd)

----------------------------------------------------------------------------------------------
 P2Rank 2.3
----------------------------------------------------------------------------------------------

predicting pockets for proteins from dataset [protein_list.ds]
processing [7kac.pdb] (1/6)
processing [7s1s.pdb] (3/6)
processing [6scm.pdb] (2/6)
processing [7t47.pdb] (4/6)
processing [6rpg.pdb] (6/6)
processing [7kcc.pdb] (5/6)
predicting pockets finished in 0 hours 0 minutes 10.648 seconds
results saved to directory [/home/jovyan/dataspace/Prototype/p2rank_results_for_internal_data]

----------------------------------------------------------------------------------------------
 finished successfully in 0 hours 0 minutes 11.918 seconds
----------------------------------------------------------------------------------------------


0

### 构建 `info` 文件

In [68]:
info = []
err_pdb_list=[]
for i, line in tqdm(data.iterrows(), total=data.shape[0]):
    
    sample_id = line['sample_id']
    pdb_id = line['pdb']
    smiles_name = line['sample_id']
    smiles = line['smiles']
    session_au = line['target']
    value = line['value']
         
    p2rankFile = f"/home/jovyan/dataspace/Prototype/p2rank_results_for_internal_data/{pdb_id}.pdb_predictions.csv"    
    pocket = pd.read_csv(p2rankFile)
    pocket.columns = pocket.columns.str.strip()
    pocket_coms = pocket[['center_x', 'center_y', 'center_z']].values

    # protein center as a block.
    protein_com = protein_dict[pdb_id][0].numpy().mean(axis=0).astype(float).reshape(1, 3)
    info.append([sample_id, pdb_id, smiles_name, smiles, session_au, protein_com, pdb_id+"_c", 
                    value,  "test"])
        
    for idx, pocket_line in pocket.iterrows():
        pdb_idx = f"{pdb_id}_{idx}"
        info.append([sample_id, pdb_id, smiles_name, smiles, session_au, pocket_coms[idx].reshape(1, 3), pdb_idx, 
                    value, "test"])
            
info = pd.DataFrame(
info, 
columns=['sample_id', 'pdb_id', 'smiles_name', 'kekule_smiles', 'session_au', 'pocket_com', 'pocket_name', 'value', 'split_tag'])
print(len(info))

info.to_csv("/home/jovyan/dataspace/Prototype/tables/INTERNAL_info.csv")
info.to_pickle("/home/jovyan/dataspace/Prototype/tables/INTERNAL_info.pkl")

  0%|          | 0/1057 [00:00<?, ?it/s]

9784


### 人工选定口袋

In [69]:
pocket_dict = {
    "7kac": "7kac_1", # or 7kac_2
    "6scm": "6scm_4",
    "7s1s": "7s1s_1",
    "6rgp": "6rpg_1",
    "7kcc": "7kcc_1", # or 7kcc_5
    "7t47": "7t47_2"
}
pocket_names = ["7kac_1", "6scm_4", "7s1s_1", "6rpg_1", "7kcc_1", "7t47_2"]

In [70]:
info = pd.read_pickle("/home/jovyan/dataspace/Prototype/tables/INTERNAL_info.pkl")
info = info[info.pocket_name.isin(pocket_names)].reset_index(drop=True)
info.to_pickle("/home/jovyan/dataspace/Prototype/tables/INTERNAL_info_sole_pocket.pkl")
info

,sample_id,pdb_id,smiles_name,kekule_smiles,session_au,pocket_com,pocket_name,value,split_tag
0,ID_internal_KRAS-G12D_0,7t47,ID_internal_KRAS-G12D_0,Oc1cc(-c2ncc3c(NC4CNC4)nc(OCC45CCCN4CCC5)nc3c2...,KRAS-G12D,"[[-26.3188, -11.827, 17.87]]",7t47_2,46.5,test
1,ID_internal_KRAS-G12D_1,7t47,ID_internal_KRAS-G12D_1,CN[C@H]1C[C@@H](Nc2nc(OCC34CCCN3CCC4)nc3c(F)c(...,KRAS-G12D,"[[-26.3188, -11.827, 17.87]]",7t47_2,80.9,test
2,ID_internal_KRAS-G12D_2,7t47,ID_internal_KRAS-G12D_2,C#Cc1cccc2cccc(-c3ncc4c(NCC5(N(C)C)CCC5)nc(OC[...,KRAS-G12D,"[[-26.3188, -11.827, 17.87]]",7t47_2,22.0,test
3,ID_internal_KRAS-G12D_3,7t47,ID_internal_KRAS-G12D_3,CCc1cccc2cccc(-c3ncc4c(NCC5(N(C)C)CCC5)nc(OC[C...,KRAS-G12D,"[[-26.3188, -11.827, 17.87]]",7t47_2,59.2,test
4,ID_internal_KRAS-G12D_4,7t47,ID_internal_KRAS-G12D_4,CCc1c(F)ccc2cccc(-c3ncc4c(NCC5(N(C)C)CCC5)nc(O...,KRAS-G12D,"[[-26.3188, -11.827, 17.87]]",7t47_2,30.4,test
...,...,...,...,...,...,...,...,...,...
1052,ID_internal_SOS1_1052,6scm,ID_internal_SOS1_1052,C[C@@H](NC(=O)c1cc(N2CCn3nccc3C2)nc2cc[nH]c12)...,SOS1,"[[7.9823, 19.0694, -15.4211]]",6scm_4,89.8,test
1053,ID_internal_SOS1_1053,6scm,ID_internal_SOS1_1053,C[C@@H](NC(=O)c1cc(N2CCc3c(cnn3C)C2)nc2cc[nH]c...,SOS1,"[[7.9823, 19.0694, -15.4211]]",6scm_4,105.5,test
1054,ID_internal_SOS1_1054,6scm,ID_internal_SOS1_1054,C[C@H](NC(=O)c1cc(N2CCC(O)C2)nc2cc[nH]c12)c1cc...,SOS1,"[[7.9823, 19.0694, -15.4211]]",6scm_4,109.6,test
1055,ID_internal_SOS1_1055,6scm,ID_internal_SOS1_1055,COC1CCN(c2cc(C(=O)N[C@H](C)c3cccc(C(F)F)c3F)c3...,SOS1,"[[7.9823, 19.0694, -15.4211]]",6scm_4,125.4,test


# <font color='gold'>「从这里开始」</font>数据集构建

In [ ]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import os
import torch
pdbbind_path = "/home/jovyan/dataspace/PDBbind2020/all_pdbbind"
proto_path = "/home/jovyan/dataspace/Prototype"
p2rank = "bash /home/jovyan/dataspace/p2rank_2.3/prank"
tankbind_src_folder_path = "../tankbind_prototype/"
import sys
sys.path.insert(0, tankbind_src_folder_path)
from feature_utils import generate_sdf_from_smiles_using_rdkit

In [71]:
info = pd.read_pickle("/home/jovyan/dataspace/Prototype/tables/INTERNAL_info_sole_pocket.pkl")
protein_dict = torch.load("/home/jovyan/dataspace/Prototype/dicts/INTERNAL_protein_dict.pt")
compound_dict = torch.load("/home/jovyan/dataspace/Prototype/dicts/INTERNAL_compound_torchdrug_features.pt")

In [72]:
from data_prototype import TankBindDataSet_prototype
toFileInternal = f"/home/jovyan/main_tankbind/dataset_prototype/internal"
os.system(f"rm -rf {toFileInternal}")
os.system(f"mkdir -p {toFileInternal}")
dataset = TankBindDataSet_prototype(toFileInternal, data=info, protein_dict=protein_dict, compound_dict=compound_dict)

Processing...
Done!


['/home/jovyan/main_tankbind/dataset_prototype/internal/processed/data.pt', '/home/jovyan/main_tankbind/dataset_prototype/internal/processed/protein.pt', '/home/jovyan/main_tankbind/dataset_prototype/internal/processed/compound.pt']


In [73]:
t = []
t_dict = {}
data = dataset.data

for i, line in tqdm(data.iterrows(), total=data.shape[0]):
    d = dataset[i]
    sample_id = line['sample_id']
    p_length = d['node_xyz'].shape[0]
    c_length = d['coords'].shape[0]
    y_length = d['y'].shape[0]
    t.append([i, sample_id, p_length, c_length, y_length])
    t_dict[sample_id] = [i, sample_id, p_length, c_length, y_length]
    
torch.save(t, "/home/jovyan/dataspace/Prototype/tables/INTERNAL_supplementary.pt")
torch.save(t_dict, "/home/jovyan/dataspace/Prototype/dicts/INTERNAL_supplementary_dict.pt")

t = pd.DataFrame(t, columns=['index', 'sample_id', 'p_length', 'c_length', 'y_length'])
t.to_csv("/home/jovyan/dataspace/Prototype/tables/INTERNAL_supplementary.csv")

data = pd.concat([data, t[['p_length', 'c_length', 'y_length']]], axis=1)
torch.save(data, f"{toFileInternal}/processed/data.pt")

  0%|          | 0/1057 [00:00<?, ?it/s]

In [ ]:
data = dataset.data
t = pd.read_csv("/home/jovyan/dataspace/Prototype/tables/INTERNAL_supplementary.csv", index_col=0)
data =pd.concat([data, t[['p_length', 'c_length', 'y_length']]], axis=1)
torch.save(data, f"{toFileInternal}/processed/data.pt")